In [39]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

from matchup_generator import generate_matchups

In [40]:
training_df = generate_matchups(5000)
training_df

/Users/anirudh/UIUC/go-mama-27/personal-projects/poke-predictor/ml-model/db.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,p1_id,p1_name,p1_hp,p1_attack,p1_defense,p1_sp_attack,p1_sp_defense,p1_speed,p1_primary_type,p1_secondary_type,p2_id,p2_name,p2_hp,p2_attack,p2_defense,p2_sp_attack,p2_sp_defense,p2_speed,p2_primary_type,p2_secondary_type,p3_id,p3_name,p3_hp,p3_attack,p3_defense,p3_sp_attack,p3_sp_defense,p3_speed,p3_primary_type,p3_secondary_type,p4_id,p4_name,p4_hp,p4_attack,p4_defense,p4_sp_attack,p4_sp_defense,p4_speed,p4_primary_type,p4_secondary_type,...,opp3_name,opp3_hp,opp3_attack,opp3_defense,opp3_sp_attack,opp3_sp_defense,opp3_speed,opp3_primary_type,opp3_secondary_type,opp4_id,opp4_name,opp4_hp,opp4_attack,opp4_defense,opp4_sp_attack,opp4_sp_defense,opp4_speed,opp4_primary_type,opp4_secondary_type,opp5_id,opp5_name,opp5_hp,opp5_attack,opp5_defense,opp5_sp_attack,opp5_sp_defense,opp5_speed,opp5_primary_type,opp5_secondary_type,opp6_id,opp6_name,opp6_hp,opp6_attack,opp6_defense,opp6_sp_attack,opp6_sp_defense,opp6_speed,opp6_primary_type,opp6_secondary_type,winner
0,128,tauros,75,100,95,40,70,110,normal,None,5,charmeleon,58,64,58,80,65,80,fire,None,63,abra,25,20,15,105,55,90,psychic,None,132,ditto,48,48,48,48,48,48,normal,None,...,venonat,60,55,50,40,55,45,bug,poison,4,charmander,39,52,43,60,50,65,fire,None,74,geodude,40,80,100,30,30,20,rock,ground,90,shellder,30,65,100,45,25,40,water,None,2
1,67,machoke,80,100,70,50,60,45,fighting,None,123,scyther,70,110,80,55,80,105,bug,flying,97,hypno,85,73,70,73,115,67,psychic,None,14,kakuna,45,25,50,25,25,35,bug,poison,...,beedrill,65,90,40,45,80,75,bug,poison,31,nidoqueen,90,92,87,75,85,76,poison,ground,102,exeggcute,60,40,80,60,45,40,grass,psychic,12,butterfree,60,45,50,90,80,70,bug,flying,1
2,48,venonat,60,55,50,40,55,45,bug,poison,125,electabuzz,65,83,57,95,85,105,electric,None,149,dragonite,91,134,95,100,100,80,dragon,flying,121,starmie,60,75,85,100,85,115,water,psychic,...,caterpie,45,30,35,20,20,45,bug,None,81,magnemite,25,35,70,95,55,45,electric,steel,146,moltres,90,100,90,125,85,90,fire,flying,52,meowth,40,45,35,40,40,90,normal,None,2
3,148,dragonair,61,84,65,70,70,70,dragon,None,42,golbat,75,80,70,65,75,90,poison,flying,141,kabutops,60,115,105,65,70,80,rock,water,143,snorlax,160,110,65,65,110,30,normal,None,...,machop,70,80,50,35,35,35,fighting,None,91,cloyster,50,95,180,85,45,70,water,ice,37,vulpix,38,41,40,50,65,65,fire,None,96,drowzee,60,48,45,43,90,42,psychic,None,1
4,17,pidgeotto,63,60,55,50,50,71,normal,flying,147,dratini,41,64,45,50,50,50,dragon,None,132,ditto,48,48,48,48,48,48,normal,None,122,mr-mime,40,45,65,100,120,90,psychic,fairy,...,venomoth,70,65,60,90,75,90,bug,poison,3,venusaur,80,82,83,100,100,80,grass,poison,111,rhyhorn,80,85,95,30,30,25,ground,rock,43,oddish,45,50,55,75,65,30,grass,poison,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,61,poliwhirl,65,65,65,50,50,90,water,None,65,alakazam,55,50,45,135,95,120,psychic,None,9,blastoise,79,83,100,85,105,78,water,None,6,charizard,78,84,78,109,85,100,fire,flying,...,exeggutor,95,95,85,125,75,55,grass,psychic,40,wigglytuff,140,70,45,85,50,45,normal,fairy,66,machop,70,80,50,35,35,35,fighting,None,89,muk,105,105,75,65,100,50,poison,None,2
4996,149,dragonite,91,134,95,100,100,80,dragon,flying,2,ivysaur,60,62,63,80,80,60,grass,poison,63,abra,25,20,15,105,55,90,psychic,None,14,kakuna,45,25,50,25,25,35,bug,poison,...,abra,25,20,15,105,55,90,psychic,None,7,squirtle,44,48,65,50,64,43,water,None,101,electrode,60,50,70,80,80,150,electric,None,107,hitmonchan,50,105,79,35,110,76,fighting,None,1
4997,72,tentacool,40,40,35,50,100,70,water,poison,150,mewtwo,106,110,90,154,90,130,psychic,None,111,rhyhorn,80,85,95,30,30,25,ground,rock,40,wigglytuff,140,70,45,85,50,45,normal,fairy,...,bulbasaur,45,49,49,65,65,45,grass,poison,41,zubat,40,45,35,30,40,55,poison,f

In [41]:
X = training_df.drop(columns=["winner"])
y = training_df["winner"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [42]:
categorical_cols = [
    'p1_primary_type', 'p1_secondary_type',
    'opp1_primary_type', 'opp1_secondary_type',
    'opp2_primary_type', 'opp2_secondary_type',
    'opp3_primary_type', 'opp3_secondary_type',
    'opp4_primary_type', 'opp4_secondary_type',
    'opp5_primary_type', 'opp5_secondary_type',
    'opp6_primary_type', 'opp6_secondary_type'
]

In [43]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat = ohe.fit_transform(X_train[categorical_cols])
X_test_cat = ohe.transform(X_test[categorical_cols])

In [44]:
numeric_cols_x = X_train.select_dtypes(include='number').columns

sc = StandardScaler()
X_train_numeric = sc.fit_transform(X_train[numeric_cols_x])
X_test_numeric = sc.transform(X_test[numeric_cols_x])

X_train_combined = np.hstack([X_train_numeric, X_train_cat])
X_test_combined = np.hstack([X_test_numeric, X_test_cat])

In [45]:
X_train_tensor = torch.tensor(X_train_combined, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_combined, dtype=torch.float32)

y_train_tensor = torch.tensor((y_train.to_numpy() - 1).astype(np.float32)).view(-1, 1)
y_test_tensor = torch.tensor((y_test.to_numpy() - 1).astype(np.float32)).view(-1, 1)

In [46]:
class LogisticRegression(nn.Module):
    def __init__(self, n):
        super(LogisticRegression, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        
    def forward(self, x):
        return self.net(x)

In [47]:
input_dim = X_train_tensor.shape[1]
model = LogisticRegression(input_dim)

In [48]:
learning_rate = 0.01
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [50]:
num_epochs = 100

for epoch in range(num_epochs):
    y_pred = model(X_train_tensor)
    loss = criterion(y_pred, y_train_tensor)
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 10, loss = 0.5849
epoch: 20, loss = 0.4548
epoch: 30, loss = 0.2667
epoch: 40, loss = 0.1128
epoch: 50, loss = 0.0394
epoch: 60, loss = 0.0157
epoch: 70, loss = 0.0082
epoch: 80, loss = 0.0053
epoch: 90, loss = 0.0040
epoch: 100, loss = 0.0032


In [53]:
model.eval()
with torch.no_grad():
    y_pred_logits = model(X_test_tensor).squeeze()
    y_pred_probs = torch.sigmoid(y_pred_logits)
    y_pred_cls = (y_pred_probs >= 0.5).float()
    acc = (y_pred_cls == y_test_tensor).float().mean()
    print(f'accuracy = {acc:.4f}')

accuracy = 0.5001
